Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/explain-model/azure-integration/remote-explanation/explain-model-on-amlcompute.png)

## Setup

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Fetch the Workspace

Connect to the workspace

In [ ]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

In [ ]:
model = ws.models['attrition_python']

In [ ]:
model.datasets

### Create an environment for Scikit Learn

In [ ]:
from azureml.core import Environment

env = Environment.from_conda_specification(name = "ScikitEnvironment", file_path='scikit.yaml')

env.register(ws)

In [ ]:
# if already created, just retrieve it
env = ws.environments['ScikitEnvironment']

### Create an Inference Config for our Model

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py", environment=env)

In [ ]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.exceptions import WebserviceException
from azureml.core import Model

deployment_config = AksWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
aks_service_name = 'attritionprod'
aks_cluster = ws.compute_targets['aks']

try:
    # if you want to get existing service below is the command
    # since aci name needs to be unique in subscription deleting existing aci if any
    # we use aci_service_name to create azure aci
    service = Webservice(ws, name=aks_service_name)
    if service:
        service.delete()
except WebserviceException as e:
    print()

service = Model.deploy(ws, aks_service_name, [model], inference_config, deployment_config, aks_cluster)



In [ ]:
service.wait_for_deployment(True)
print(service.state)

In [ ]:
# if there are any issues whatsoever, there first place to look is here
print(service.get_logs())

In [ ]:
import pandas as pd
df = pd.read_csv('../data/IBM-Employee-Attrition.csv')

In [ ]:
service.run(df.iloc[0:5].to_json())